In [1]:
import urllib.request, urllib.error
import requests
from bs4 import BeautifulSoup
from fake_headers import Headers
import time
import pandas as pd
import os
import re

In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

with open("your_ID.txt") as f:
    l = f.readlines()

client_id=l[0].strip()
client_secret = l[1].strip()
username = l[2].strip()

token = util.prompt_for_user_token(username=username, scope='user-read-playback-state,playlist-read-private,user-modify-playback-state,playlist-modify-public', 
                                   client_id = client_id,client_secret = client_secret,redirect_uri="http://localhost:8889/callback")

sp = spotipy.Spotify(auth=token)

In [3]:
print("client_id:%s" %client_id)
print("client_secret:%s" %client_secret)
print("username:%s" %username)

client_id:55e915e6dc2549b29feb68ef4e611e2a
client_secret:086669968e7e4892bc08c8d39308afaf
username:21ulbu67tca2mhvh2jiy3g4da


In [4]:
def get_soup(url):
    """"Retrieve html and return a bs4-object."""
    response = requests.get(url, headers=Headers().generate())
    soup = BeautifulSoup(response.text, "html.parser")
    if "Error 403" in soup.title.text:
        del soup
        raise Exception("Error 403: Captcha? https://www.1001tracklists.com/")
    else:
        return soup

In [5]:
url = "https://www.officialcharts.com/charts/singles-chart/" 

In [6]:
soup = get_soup(url)

In [7]:
date = soup.find('p', class_="article-date")
date = list(date)[0]
date = date.strip('\r\n').strip()
playlist_name = "UK Official Singles Chart Top 100 (%s)" %date

In [8]:
columns=["track_url", "track_name", "track_artist_url", "track_artist_name"]
df = pd.DataFrame(index=[], columns=columns)

In [9]:
track = soup.find_all('div', class_="title-artist")[0]

In [10]:
i = 0
for track in soup.find_all('div', class_="title-artist"):
    text = track.find_all("a")
    track_url = text[0].get("href")
    track_name = list(text[0])[0]
    track_artist_url = text[1].get("href")
    track_artist_name = list(text[1])[0]
    df.loc[i,"track_url"] = track_url
    df.loc[i,"track_name"] = track_name
    df.loc[i,"track_artist_url"] = track_artist_url
    df.loc[i,"track_artist_name"] = track_artist_name
    i = i +1

In [11]:
track_name,artist_name  = df.loc[0,["track_name","track_artist_name"]]

In [12]:
def get_id_track_artist_2(track_name,artist_name):
    
    id_list = []
    name_list = []
    artists = []
    result = sp.search(q='track:'+track_name, limit=10, offset=0, type='track', market=None)
    for track in result['tracks']['items']:
        if track["artists"][0]['name'].lower() == artist_name.lower():
            id_list.append(track['id'])
            name_list.append(track["name"])
            artists.append(track["artists"][0]['name'])

    l = pd.DataFrame({'id': id_list ,'track': name_list, "artist" : artists})

    if len(l) ==0:
        for track in result['tracks']['items']:
            if (track["artists"][0]['name'].lower() in artist_name.lower()) or (artist_name.lower() in track["artists"][0]['name'].lower()):
                id_list.append(track['id'])
                name_list.append(track["name"])
                artists.append(track["artists"][0]['name'])

    l = pd.DataFrame({'id': id_list ,'track': name_list, "artist" : artists})

    return l

In [13]:
def get_id_track_artist(track_name,artist_name):
    id_list = []
    name_list = []
    artists = []
    result = sp.search(q='artist:'+artist_name.split("/")[0], limit=10, offset=0, type='track', market=None)
    for track in result['tracks']['items']:
        #print(track['name'].lower())
        if track['name'].lower() == track_name.lower():
            id_list.append(track['id'])
            name_list.append(track["name"])
            artists.append(track["artists"][0]['name'])

    l = pd.DataFrame({'id': id_list ,'track': name_list, "artist" : artists})

    if len(l) ==0:
        for track in result['tracks']['items']:
            if (track['name'].lower() in artist_name.lower()) or (artist_name.lower() in track['name'].lower()):
                id_list.append(track['id'])
                name_list.append(track["name"])
                artists.append(track["artists"][0]['name'])

    l = pd.DataFrame({'id': id_list ,'track': name_list, "artist" : artists})
    
    if len(l) ==0:
        l = get_id_track_artist_2(track_name,artist_name)
        
    if len(l) ==0:
        l = pd.DataFrame({"id":["None"], "track":["None"],"artist":["None"]})
        print("this track is None:%s/%s" %(track_name,artist_name))
    return l.iloc[0,:]


In [14]:
cols = ['id', 'track',"artist"]
play_list_df = pd.DataFrame(index=[], columns=cols)
for i in range(len(df)):
    track_name,artist_name  = df.loc[i,["track_name","track_artist_name"]]
    l = get_id_track_artist(track_name,artist_name)
    play_list_df = play_list_df.append(pd.DataFrame(l).T, ignore_index=True )

this track is None:READY/FREDO FT SUMMER WALKER
this track is None:FRIDAY/RITON/NIGHTCRAWLERS/MUFASA
this track is None:LOVE NOT WAR (THE TAMPA BEAT)/JASON DERULO & NUKA
this track is None:BE THE ONE/RUDIMENTAL/MORGAN/DIGGA D/TIKE
this track is None:FOREVER YOUNG/BECKY HILL


In [15]:
#rank of None tracks
print(play_list_df[play_list_df["id"] == "None"])

      id track artist
20  None  None   None
23  None  None   None
31  None  None   None
57  None  None   None
84  None  None   None


In [16]:
play_list_df = play_list_df[play_list_df["id"] != "None"]

In [17]:
playlists = sp.user_playlist_create(username, playlist_name, public=True, collaborative=False, description="This playlists corespond to Official Singles Chart Top 100")

playlists = sp.current_user_playlists()
for i, item in enumerate(playlists['items']):
    if item['name'] == playlist_name:
        playlist_id = item['id']

track_ids = play_list_df["id"]
sp.user_playlist_add_tracks(username, playlist_id, track_ids)

{'snapshot_id': 'MywyMjIzODFlNTFlZGYzMDkxNmQ4YzUwYTE4OGIzY2YyMTNmYTJjZDc1'}